In [0]:
!rm -rf diaretdb0_v_1_1/

In [0]:
import glob
import zipfile
import shutil
from tqdm import tqdm
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [62]:
if not os.path.exists('diaretdb0_v_1_1.zip'):
  !wget http://www.it.lut.fi/project/imageret/diaretdb0/diaretdb0_v_1_1.zip
  
zip_ref = zipfile.ZipFile('diaretdb0_v_1_1.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

--2019-03-09 05:20:22--  http://www.it.lut.fi/project/imageret/diaretdb0/diaretdb0_v_1_1.zip
Resolving www.it.lut.fi (www.it.lut.fi)... 157.24.20.41
Connecting to www.it.lut.fi (www.it.lut.fi)|157.24.20.41|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 269834063 (257M) [application/zip]
Saving to: ‘diaretdb0_v_1_1.zip’

diaretdb0_v_1_1.zip 100%[===================>] 257.33M  23.5MB/s    in 12s     

2019-03-09 05:20:35 (21.5 MB/s) - ‘diaretdb0_v_1_1.zip’ saved [269834063/269834063]



In [0]:
mask_dir = './diaretdb0_v_1_1/resources/images/diaretdb0_binary_masks'
fundus_dir = './diaretdb0_v_1_1/resources/images/diaretdb0_fundus_images'

In [0]:
def color_normalize(image):
  
  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  b = img.copy()
  # set green and red channels to 0
  b[:, :, 1] = 0
  b[:, :, 2] = 0

  g = img.copy()
  # set blue and red channels to 0
  g[:, :, 0] = 0
  g[:, :, 2] = 0

  r = img.copy()
  # set blue and green channels to 0
  r[:, :, 0] = 0
  r[:, :, 1] = 0
  
  img = (b/b.max() + g/g.max() + r/r.max())
  img = cv2.resize(img, (512, 512)) 
  img = (img * 255).astype(np.uint8)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  #
  # applying mask to source image - region segmentation
  #
  mask = cv2.imread('{0}/{1}_valid.png'.format(mask_dir, os.path.splitext(os.path.basename(image))[0]), 0)
  mask = cv2.resize(mask, (512, 512)) 
  
  result = cv2.bitwise_and(img, img, mask = mask)
  
  cv2.imwrite("./normalised_images/norm_"+os.path.basename(image), result)

In [0]:
def build_model():

  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(512, 512)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.50))

  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.50))

  model.add(Dense(1024, activation='relu'))

  model.add(Dense(5, activation='softmax'))

return model

In [60]:
if __name__ == "__main__":
  
  #
  # get image names
  #
  images = [i for i in glob.glob("{0}/image*.png".format(fundus_dir))]
  normalised = list()
  
  if not os.path.exists('normalised_images'):
    os.makedirs('normalised_images')
  else:
    shutil.rmtree('normalised_images')
    os.makedirs('normalised_images')
    
  #
  # perform color normalisation
  #
  for image in tqdm(images):
    color_normalize(image)
    normalised.append("./norm_"+os.path.basename(image)) 
  
  print("\nPre-processing complete...")

100%|██████████| 130/130 [00:20<00:00,  6.44it/s]


Pre-processing complete...
